In [1]:
import re
import os
import json
import random
import selenium
import pandas as pd
from time import sleep
from selenium import webdriver
import undetected_chromedriver as uc
from IPython.display import clear_output
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC

In [2]:
#Converting panda's dataframe to list
df = pd.read_csv('all_links.csv', header=None)
links = []

for i in range(len(df)):
    links.append(df[0][i])

In [3]:
##Function that generate request delay 
def give_me_random_number():
    return int(random.uniform(121, 140))

## Function the find digits in other information
def has_digits(string):
    return bool(re.search(r'\d', string))

In [4]:
driver = uc.Chrome()
driver.maximize_window()

# Product Title

In [5]:
def giveProductName(driver):
    product_name = ''
    try:
        product_name = driver.find_element(By.XPATH, '//div[@class="sc-pc-title-full-desktop"]//h1').text
        return product_name
    except:
        return product_name

# Other Information

In [6]:
# Extract Other Info
def giveOtherInformation(driver):
    other = {}
    other['owner'] = ''
    other['model_number'] = ''
    other['item_number'] = ''
    try:
        other['item_number'] = driver.find_element(By.XPATH, "//div[@class='sc-product-header-item-number']").text
    except:
        other['item_number'] = "####"

    try:
        product_detailes = driver.find_elements(By.XPATH, "//span[@class='sc-product-header-item-number']")
        for p in product_detailes:
            if (has_digits(p.text)):
                other['model_number'] = p.text
            else:
                other['owner'] = p.text
    except:
        other['model_number'] = "NOT"
        other['owner'] = "NOT"
    return other

# Prices

In [7]:
def givePriceDetailes(driver):
    per_unit_price = ''
    currency = ''
    whole_price = ''
    price_mantisa = ''
    try:
        per_unit_price = driver.find_element(By.XPATH, "//div[@class='sc-pc-channel-price']//div/span[2][@class='sc-pc-single-price-inline-price']").text  ## $0.94/ea
        currency = driver.find_element(By.CLASS_NAME, "Price-currency").text                         ## $
        whole_price = driver.find_element(By.XPATH, "//div[@class='sc-pc-channel-price']//span[@class='Price-characteristic']").text                ## 22
        price_mantisa = driver.find_element(By.XPATH, "//div[@class='sc-pc-channel-price']//span[@class='Price-mantissa']").text                    ## 54
        return per_unit_price, currency+whole_price+'.'+price_mantisa
    except:
        return per_unit_price, currency+whole_price+'.'+price_mantisa        

# Scrap All main and sub images

In [8]:
def giveImagesLinks(driver):
    images = []
    try:
        sub_images = driver.find_elements(By.XPATH, "//img[@class='sc-carousel-child-image-thumb sc-image-wrapper-full-res sc-image-wrapper-full-res-loaded']")
        main_image = driver.find_element(By.XPATH, "//img[@class='sc-image-viewer-img sc-viewer-zoom-img']").get_attribute("src")
        images.append(main_image)
        for image in sub_images:
            images.append(image.get_attribute("src"))  
        return images
    except:
        return images

# Scrap Categories 

In [9]:
def giveCategories(driver):
    product_category = []
    try:
        categories = driver.find_elements(By.XPATH, "//li[@property='itemListElement']//a")
        for category in categories:
            product_category.append(category.text)
        product_category.pop(0)
        return product_category
    except:
        return product_category

# Heighlishgsts

In [10]:
def giveHighlights(driver):
    highlights = []
    try:
        all_highlights = driver.find_elements(By.XPATH, "//div[@class='sc-pc-about-this-item']//div//ul//li")
        for highlight in all_highlights:
            highlights.append(highlight.text)
        ## Having different path
        if len(highlights) == 0:
            all_highlights = driver.find_elements(By.XPATH, "//div[@class='sc-pc-about-this-item']//div//li")
            for highlight in all_highlights:
                highlights.append(highlight.text) 
        ## if having not ul or li
        if len(highlights) == 0:
            all_highlights = driver.find_elements(By.XPATH, "//div[@class='sc-pc-about-this-item']//div")
            highlights.append(all_highlights.text)
        return highlights
    except:
        return highlights

# Description

In [11]:
def giveDescription(driver):
    description = ''
    try:
        description = driver.find_element(By.CLASS_NAME, "sc-description-about-long").text
        return description
    except:
        return description

# Counter

In [12]:
def startCounter():
    delay = give_me_random_number()
    for j in range(delay):
        print(delay, " -:- ",j+1)
        sleep(1)
        if j < delay - 2:
            clear_output(wait=True)

# Main Part

In [ ]:
final_data = []

##This code is use to find the data_in_json.json file 
## If it exest it will append fila_data from previous data and generate new list of scrapped data 
## AND WILL SKIPP ALL LINKS THAT WERE SCRAPPED
scrapped_url = []
file_path = os.path.join("./", "data_in_json.json")  # join folder and file name
file_exists = os.path.exists(file_path)
if file_exists:
    with open('data_in_json.json', 'r') as file:
        data = json.load(file)
    for item in data:
        final_data.append(item)
        scrapped_url.append(item["product_url"])
else:
    print("No data_in_json.json file..")


for i, link in enumerate(links):
    if i == 0 or link in scrapped_url:
        continue
    else:
        driver.get(link)
        startCounter()
        print(link)
        
        product_name = giveProductName(driver)
        per_unit_price, price = givePriceDetailes(driver)
        highlights = giveHighlights(driver)
        images = giveImagesLinks(driver)
        description = giveDescription(driver)
        product_category = giveCategories(driver)
        other = giveOtherInformation(driver)

        json_data = {
            "product_name": product_name,
            "product_price_per_unit": per_unit_price,
            "total_price": price,
            "product_highlights": highlights,
            "product_detail": description,
            "product_images": images,
            "product_url": link,
            "product_category": product_category,
            "more": other
        }
        final_data.append(json_data)
        json_string = json.dumps(final_data)
        with open('data_in_json.json', 'w') as file:
            file.write(json_string)

135  -:-  22
